<a href="https://www.kaggle.com/code/cmosbattery/mammal-classification-with-cnn-and-regularization?scriptVersionId=219007577" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Mammal Classification with CNN and Regularization

In this Notebook, we will develop a Convolutional Neural Network (CNN) model that trains on an image dataset of mammals belonging to 45 classes. The dataset used came from the Mammals Image Classification Dataset (45 Animals). We will also apply model regularization techniques to address overfitting and improve the generalization of the model.

## 1. Setup

In this section, we set up the environment and import the necessary libraries for building the CNN model, as well as for visualizing the data and handling the dataset. We will use TensorFlow for the CNN model and Matplotlib for data visualization.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np
import PIL
import os
import random
import shutil
import zipfile
from collections import defaultdict

## 2. Data Preprocessing

In this section, we preprocess the dataset to ensure that the images are ready for model training. This includes organizing the data, balancing the number of sample images across all classes, applying necessary transformations such as resizing, and splitting the dataset into training and validation sets for optimal model evaluation.

In [ ]:
data_directory = Path("/kaggle/input/mammals-image-classification-dataset-45-animals/mammals")
image_count = len(list(data_directory.rglob('*.*')))
print(f"The number of images present in the dataset: {image_count}")

In [ ]:
# Dictionary to store the count of images per class
image_count_per_class = defaultdict(int)

# Iterate through each class directory
for class_dir in data_directory.iterdir():
    if class_dir.is_dir():  # Check if it's a directory
        class_name = class_dir.name
        image_count_per_class[class_name] = len(list(class_dir.glob('*.jpg')))

# Prepare data for plotting
class_names = list(image_count_per_class.keys())
image_counts = list(image_count_per_class.values())

plt.figure(figsize=(9, 6))
plt.barh(class_names, image_counts)
plt.title(f"Image Count per Class ({len(image_count_per_class)} Mammal Classes)")
plt.xlabel("Number of Images")
plt.ylabel("Class Names")
plt.tight_layout()
plt.show

Some of the classes have greater amount of images than others, this creates data imbalance and could potentially cause bias during the training process. This results for them have better representation during the training than those who have few images. With that, we will only limit them to only having 200 images per class.

In [ ]:
REQ_IMAGE_COUNT = 200

# Create a list to store the filtered images
filtered_images = []

# Dictionary to store the new count of images per class after filtering
filtered_image_count_per_class = defaultdict(int)

# Iterate through each class directory
class_directories = [d for d in data_directory.iterdir() if d.is_dir()]

for class_dir in class_directories:
    images = list(class_dir.glob('*.jpg'))
    
    # If more than 200 images are present, randomly select 200
    if len(images) > REQ_IMAGE_COUNT:
        images = random.sample(images, REQ_IMAGE_COUNT)

    class_name = class_dir.name
    filtered_image_count_per_class[class_name] = len(images)
    
    # Add the selected images to the filtered list
    filtered_images.extend(images)

# Prepare data for plotting
class_names = list(filtered_image_count_per_class.keys())
image_counts = list(filtered_image_count_per_class.values())

plt.figure(figsize=(9, 6))
plt.barh(class_names, image_counts)
plt.title(f"Image Count per Class")
plt.xlabel("Number of Images")
plt.ylabel("Class Names")
plt.tight_layout()
plt.show


# Check the total number of images collected
print(f"Total images collected: {len(filtered_images)}")

We will be storing these filtered classes into a new directory.

> directory: /kaggle/working/filtered_mammals 

In [ ]:
# Create a new directory to store the filtered dataset
filtered_data_directory = Path("/kaggle/working/filtered_mammals")

# Create subdirectories for each class
for class_dir in class_directories:
    class_name = class_dir.name
    (filtered_data_directory / class_name).mkdir(parents=True, exist_ok=True)

# Copy filtered images to the new directory
for image_path in filtered_images:
    class_name = image_path.parent.name
    destination = filtered_data_directory / class_name / image_path.name
    shutil.copy(image_path, destination)

We split the dataset for training, which accounts to 80% of the whole dataset, and 20% for validation.

In [ ]:
IMAGE_BATCH = 32
IMAGE_SIZE = (150, 150)
VAL_SPLIT = 0.2

train_dataset = tf.keras.utils.image_dataset_from_directory(
    filtered_data_directory,
    validation_split=VAL_SPLIT,
    subset="training",
    seed=123,
    image_size=(IMAGE_SIZE),
    batch_size=IMAGE_BATCH,
    label_mode='int'
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    filtered_data_directory,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=123,
    image_size=(IMAGE_SIZE),
    batch_size=IMAGE_BATCH,
    label_mode='int'
)

We will do a final check on the number of classes present in the training dataset below which is indeed 45 classes.

In [ ]:
class_names = train_dataset.class_names
print('Number of classes:', len(class_names), "\n", class_names)

We will visually check the images contained in the classes of the training dataset.

In [ ]:
plt.figure(figsize=(15, 10))
for images, labels in train_dataset.take(1):
  for i in range(15):
    ax = plt.subplot(3, 5, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(buffer_size=1000).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

Prefetching is done in order to optimize data loading and improve the training time efficiency.

## 3. Building and Training the Baseline Model

For the initial baseline model, we will train the CNN with a standard configuration of three convolutional layers. The model will run for 20 epochs to observe how well the baseline performs with simple architecture. The model will be validated on the separate validation subdataset to assess its performance and observe any overfitting. 

In [ ]:
num_classes = len(class_names)

model = Sequential([
    layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    layers.Rescaling(1./255),

    # First layer with 16 filters
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    # Second layer with 32 filters
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),

    # Third layer with 64 filters
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.summary()

Before training the model, we compile it with the Adam optimizer, which adapts the learning rate for efficient training. The Sparse Categorical Crossentropy loss function is used since the task is a multi-class classification problem with integer labels. The model will also track accuracy as a performance metric during training

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20
)

In [ ]:
# Function to visualize the training and validation logs from model's history
def vis_logs(model):
    acc = model.history['accuracy']
    val_acc = model.history['val_accuracy']
    
    loss = model.history['loss']
    val_loss = model.history['val_loss']
    
    epochs_range = range(len(acc))
    
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel("Completed Epochs")
    plt.ylabel("Accuracy")
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel("Completed Epochs")
    plt.ylabel("Loss")
    plt.show()

vis_logs(history)

After training for 20 epochs, we observe significant gaps between the accuracy and loss in both plots. This is a clear indication of **overfitting** wherein the model performs too well in the training data, however in the validation, it is not performing well on the unforseen data. 

To address this issue of overfitting of our baseline model, we plan to implement the following techniques:

- **Data Augmentation**: Use data augmentation to artificially increase the size of the training set, providing the model with more varied examples to learn from.
- **Dropout**: Introduce dropout layers to randomly deactivate neurons during training, forcing the model to learn more robust features.
- **L2 Regularization**: Apply L2 regularization to penalize large weights and encourage the model to generalize better.

Additionally, we will implement the early stopping mechanism to halt training when the validation loss stops improving and adjust the learning rate if the validation performance plateaus during training.

## 4. Data Augmentation

The training images will be randomly augmented both horizontally and vertically with random rotation and zooming. This allows the model in trainig session to learn all the possible features even if the images are rotated or zoomed. This augmentation approach will be configured as a layer to the model.

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomFlip(mode="horizontal_and_vertical"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_dataset.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

After we define the data augmentation layer, we will insert it in between the input (layers.Input) layer and the normalization (layers.Rescaling) layer.

In [ ]:
num_classes = len(class_names)

model_2 = Sequential([
    layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    data_augmentation, 
    layers.Rescaling(1./255),

    # First layer with 16 filters
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    # Second layer with 32 filters
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),

    # Third layer with 64 filters
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model_2.summary()



For this training, we will train the model for 20 epochs and set the early stopping mechanism with a patience of **5 epochs**, which will stop training if the validation loss does not improve for **5 consecutive epochs**. Additionally, we will adjust the learning rate by reducing it by **50%** if the validation loss plateaus, with a patience of **3 epochs** and a minimum learning rate of **1e-6**.

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model_2.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=3, 
    min_lr=1e-6
)

history_2 = model_2.fit(
  train_dataset,
  validation_data=val_dataset,
  epochs=20,
  callbacks=[early_stopping, reduce_lr]
)

In [ ]:
vis_logs(history_2)

After training the model with data augmentation for **20 epochs**, we observe that both the training and validation accuracy and loss improved over time, showing some reduction in overfitting. As the training process reached the end of the epoch, the learning rate further decreased to **5x10^-5**, indicating that at the later stages of the training a degradation in validation performance is being observed.

Moving forward, to further address the overfitting issue, we will apply regularization techniques, such as dropout, with the goal of enhancing the model's performance on the validation data. We will also increasing the minimum number of filters to **32** from the initial value of **16**.

## 5. Model Regularization: Dropout

We will add a **Dropout layer** with a rate of **10%** after the **Dense layer** to help model reduce the overfitting by randomly deactivating the neurons during the training process.

In [ ]:
num_classes = len(class_names)

model_3 = Sequential([
    layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    data_augmentation, 
    layers.Rescaling(1./255),

    # First layer with 32 filters
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    # Second layer with 64 filters
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),

    # Third layer with 128 filters
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(num_classes, activation='softmax')
])

model_3.summary()

For this training, we compile the model with the newly added Dropout layer and an updated learning rate of 1x10^-5. The model will be trained for the same number of epochs as the previous training.

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)

model_3.compile(optimizer=optimizer,
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=3, 
    min_lr=1e-8
)

history_3 = model_3.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
vis_logs(history_3)

After training, we observe a gradual reduction in both training and validation loss. Additionally, the validation accuracy shows an upward trend, getting closer to the training accuracy, indicating that the model is learning the patterns in the data while managing to reduce overfitting. The addition of **Dropout rate** of **10%** is helping in improving generalization of the model during the validation process.

## 5. Model Regularization: L2 Regularization

To further improve the generalization of the model and reduce the risk of overfitting, we will be implementing **L2 regularization** across all convolutional layers. We apply an **L2 penalty** of **0.01** to each convolutional layer using the **kernel_regularizer** parameter. This adjustment aims to strike a balance between model flexibility and stability during training.

In [ ]:
num_classes = len(class_names)
l2 = 0.01

model_4 = Sequential([
    layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    data_augmentation, 
    layers.Rescaling(1./255),

    # First layer with 32 filters
    layers.Conv2D(32, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l2)),
    layers.MaxPooling2D(),
    
    # Second layer with 64 filters
    layers.Conv2D(64, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l2)),
    layers.MaxPooling2D(),

    # Third layer with 128 filters
    layers.Conv2D(128, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l2)),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(num_classes, activation='softmax')
])

model_4.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model_4.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=5,
    )
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=3, 
    min_lr=1e-6
)


history_4 = model_4.fit(
  train_dataset,
  validation_data=val_dataset,
  epochs=20,
  callbacks=[early_stopping, reduce_lr]
)

In [ ]:
vis_logs(history_4)

After training, both validation accuracy and loss showed consistent improvement, with the model achieving a final validation accuracy of **16.06%** and validation loss of **3.3828**. L2 regularization effectively addressed overfitting by penalizing large weight values, leading to a more generalized model. The steady decline in validation loss indicates that the model continues to learn meaningful features without overfitting.

## 6. Conclusion

We are able to develop a Convolutional Neural Network model that is trained on the [Mammals Image Classification Dataset (45 Animals)](https://www.kaggle.com/datasets/asaniczka/mammals-image-classification-dataset-45-animals) with **45 classes**. As we build the baseline model with standard configuration of three convolutional layers, we have observed an overfitting issue, with the training accuracy reaching as high as **98%** while validation accuracy remained around **~20%**. To address the issue and improve the performance, we implemented **data augmentation**, **dropout**, and **L2 regularization** which helped improve generalization and stabilize the model's performance. 

Given the dataset size of only **200 images per class**, further increasing the number of epochs did not yield significant improvements in performance. The model achieved stability, indicating that the current dataset size might limit further accuracy gains.